In [7]:
%pylab
from glob import glob
from natsort import natsorted
import h5py
from scipy.stats import binned_statistic
import palettable.colorbrewer.qualitative
from matplotlib.ticker import ScalarFormatter, NullFormatter,FormatStrFormatter

from palettable.colorbrewer.qualitative import Dark2_7
#from palettable.cubehelix import Cubehelix
#palette = Cubehelix.make(n=10)

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/mike/.local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# big multi-panel plot

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,4,figsize=(8,4))
fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

plt.rcParams.update({'axes.labelsize': 6})
#plt.rcParams.update({'axes.ticklabelsize': 8})

plt.rc('xtick',labelsize=6)
plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    

    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
    #            if not (mstar>0.08).sum(): continue
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())

            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            machs.append(np.array(F["Mach"]))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    print(mtot.max())
    #ls = ('solid' if i < 7 else 'dashed')
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
        
    times -= times[Nsink>1][0]
    ax[0,0].plot(times, mtot/M0, label=labeldict[name], color=color,ls=ls,zorder=z,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1,ls=ls,zorder=z,color=color)
    ax[0,2].plot(mtot/M0, mquantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
    ax[1,1].plot(mtot/M0, quantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
    ax[1,2].plot(mtot/M0, means,lw=1,ls=ls,zorder=z,color=color)
    ax[1,3].plot(mtot/M0, quantiles[:,-1],lw=1,ls=ls,zorder=z,color=color)
    ax[1,0].plot(times, machs,zorder=z,lw=1,ls=ls,color=color)
    i += 1
Tmax=12
ax[0,0].set(yscale='log',xlim=[0.1,Tmax],ylim=[3e-4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')
ax[0,1].set(yscale='log',xscale='log',ylim=[1,8000], xlim=[0.1,Tmax],ylabel="$N_\mathrm{*}$",xlabel=r"$t$ (Myr)")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.8,130], xlim=[1e-3,0.3], ylabel="$M_\mathrm{50}\,(M_\odot)$",xlabel=r"SFE")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.03,10], xlim=[1e-3,0.3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel=r"SFE")
ax[1,2].set(yscale='log',xscale='log',ylim=[0.2,30], xlim=[1e-3,0.3],ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel=r"SFE")
ax[1,3].set(yscale='log',xscale='log',ylim=[1,1000], xlim=[1e-3,0.3], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel=r"SFE")
ax[1,0].set(yscale='log',xscale='log', xlim=[1,Tmax], ylim=[7,30],xlabel=r"$t$ (Myr)",ylabel="$\mathcal{M}$")

ax[0,0].legend(labelspacing=0,fontsize=7,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE_IMF.pdf",bbox_inches='tight')

## Just SF histories

In [4]:
plt.clf()
fig, ax = plt.subplots(1,1,figsize=(4,4))
#fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])


ax.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

#plt.rcParams.update({'axes.labelsize': 6})
#plt.rcParams.update({'axes.ticklabelsize': 8})

#plt.rc('xtick',labelsize=6)
#plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    

    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
    #            if not (mstar>0.08).sum(): continue
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)
            mtot.append(mstar.sum())
            Nsink.append(len(mstar))
            times.append(F["Header"].attrs["Time"]*979)
    
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
        
    times -= times[Nsink>1][0]
    ax.plot(times, mtot/M0, label=labeldict[name], ls=ls,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    i += 1
Tmax=12
ax.set(yscale='log',xlim=[0.1,Tmax],ylim=[1/2e4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')

ax.legend(labelspacing=0,fontsize=7)#,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

#fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE.pdf",bbox_inches='tight')

sinkdata/b_default_n_n/stars
sinkdata/b_default_n_y/stars
sinkdata/b_default_y_n/stars
sinkdata/b_default_y_y/stars
sinkdata/box/stars
sinkdata/box_jets/stars
sinkdata/sphere/stars
sinkdata/sphere_jets/stars


# IMF

In [16]:
plt.clf()
fig, ax = plt.subplots(1,1,figsize=(4,4))
#fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])


ax.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue

    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    

    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
    #            if not (mstar>0.08).sum(): continue
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)                
            mtot.append(mstar.sum())

            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))

    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
        
    times -= times[Nsink>1][0]
    print(times,mtot)
    ax.plot(times, mtot/M0, label=labeldict[name], ls=ls,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    i += 1
Tmax=12
ax.set(yscale='log',xlim=[0.1,Tmax],ylim=[1/2e4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')

ax.legend(labelspacing=0,fontsize=7,loc=2)#,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

#fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE.pdf",bbox_inches='tight')

sinkdata/b_default_n_n/stars
[-1.90317826 -1.87846166 -1.85374505 -1.82902845 -1.80431185 -1.77959525
 -1.75487865 -1.73016205 -1.70544545 -1.68072885 -1.65601225 -1.63129565
 -1.60657905 -1.58186245 -1.55714585 -1.53242925 -1.50771264 -1.48299604
 -1.45827944 -1.43356284 -1.40884624 -1.38412964 -1.35941304 -1.33469644
 -1.30997984 -1.28526324 -1.26054664 -1.23583004 -1.21111344 -1.18639683
 -1.16168023 -1.13696363 -1.11224703 -1.08753043 -1.06281383 -1.03809723
 -1.01338063 -0.98866403 -0.96394743 -0.93923083 -0.91451423 -0.88979763
 -0.86508103 -0.84036442 -0.81564782 -0.79093122 -0.76621462 -0.74149802
 -0.71678142 -0.69206482 -0.66734822 -0.64263162 -0.61791502 -0.59319842
 -0.56848182 -0.54376522 -0.51904862 -0.49433201 -0.46961541 -0.44489881
 -0.42018221 -0.39546561 -0.37074901 -0.34603241 -0.32131581 -0.29659921
 -0.27188261 -0.24716601 -0.22244941 -0.19773281 -0.17301621 -0.1482996
 -0.123583   -0.0988664  -0.0741498  -0.0494332  -0.0247166   0.
  0.0247166   0.0494332   0.074

[-2.12562766 -2.10091106 -2.07619446 -2.05147786 -2.02676126 -2.00204466
 -1.97732806 -1.95261146 -1.92789486 -1.90317826 -1.87846166 -1.85374505
 -1.82902845 -1.80431185 -1.77959525 -1.75487865 -1.73016205 -1.70544545
 -1.68072885 -1.65601225 -1.63129565 -1.60657905 -1.58186245 -1.55714585
 -1.53242925 -1.50771264 -1.48299604 -1.45827944 -1.43356284 -1.40884624
 -1.38412964 -1.35941304 -1.33469644 -1.30997984 -1.28526324 -1.26054664
 -1.23583004 -1.21111344 -1.18639683 -1.16168023 -1.13696363 -1.11224703
 -1.08753043 -1.06281383 -1.03809723 -1.01338063 -0.98866403 -0.96394743
 -0.93923083 -0.91451423 -0.88979763 -0.86508103 -0.84036442 -0.81564782
 -0.79093122 -0.76621462 -0.74149802 -0.71678142 -0.69206482 -0.66734822
 -0.64263162 -0.61791502 -0.59319842 -0.56848182 -0.54376522 -0.51904862
 -0.49433201 -0.46961541 -0.44489881 -0.42018221 -0.39546561 -0.37074901
 -0.34603241 -0.32131581 -0.29659921 -0.27188261 -0.24716601 -0.22244941
 -0.19773281 -0.17301621 -0.1482996  -0.123583   -0

[-2.15034426 -2.07619446 -2.00204466 -1.92789486 -1.85374505 -1.77959525
 -1.70544545 -1.63129565 -1.55714585 -1.48299604 -1.40884624 -1.33469644
 -1.26054664 -1.18639683 -1.11224703 -1.03809723 -0.96394743 -0.88979763
 -0.81564782 -0.74149802 -0.66734822 -0.59319842 -0.51904862 -0.44489881
 -0.37074901 -0.29659921 -0.22244941 -0.1482996  -0.0741498   0.
  0.0741498   0.1482996   0.22244941  0.29659921  0.37074901  0.44489881
  0.51904861  0.59319842  0.66734822  0.74149802  0.81564782  0.88979763
  0.96394743  1.03809723  1.11224703  1.18639683  1.26054664  1.33469644
  1.40884624  1.48299604  1.55714585  1.63129565  1.70544545  1.77959525
  1.85374505  1.92789486  2.00204466  2.07619446  2.15034426  2.22449407
  2.29864387  2.37279367  2.44694347  2.52109327  2.59524308  2.66939288
  2.74354268  2.81769248  2.89184228  2.96599209  3.04014189  3.11429169
  3.18844149  3.2625913   3.3367411   3.4108909   3.4850407   3.5591905
  3.63334031  3.70749011  3.78163991  3.85578971  3.92993952

[-1.63129565 -1.60657905 -1.58186245 -1.55714585 -1.53242925 -1.50771264
 -1.48299604 -1.45827944 -1.43356284 -1.40884624 -1.38412964 -1.35941304
 -1.33469644 -1.30997984 -1.28526324 -1.26054664 -1.23583004 -1.21111344
 -1.18639683 -1.16168023 -1.13696363 -1.11224703 -1.08753043 -1.06281383
 -1.03809723 -1.01338063 -0.98866403 -0.96394743 -0.93923083 -0.91451423
 -0.88979763 -0.86508103 -0.84036442 -0.81564782 -0.79093122 -0.76621462
 -0.74149802 -0.71678142 -0.69206482 -0.66734822 -0.64263162 -0.61791502
 -0.59319842 -0.56848182 -0.54376522 -0.51904862 -0.49433201 -0.46961541
 -0.44489881 -0.42018221 -0.39546561 -0.37074901 -0.34603241 -0.32131581
 -0.29659921 -0.27188261 -0.24716601 -0.22244941 -0.19773281 -0.17301621
 -0.1482996  -0.123583   -0.0988664  -0.0741498  -0.0494332  -0.0247166
  0.          0.0247166   0.0494332   0.0741498   0.0988664   0.123583
  0.1482996   0.17301621  0.19773281  0.22244941  0.24716601  0.27188261
  0.29659921  0.32131581  0.34603241  0.37074901  0.39